In [ ]:
!pip install transformers
!pip install Sastrawi
!pip install Keras-Preprocessing
!pip install tensorflow keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import save_model
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical
import keras.backend as K
import pandas as pd
import re
import string
from tqdm import tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
df1 = 'https://raw.githubusercontent.com/Zaniiiii/DataGemas/main/Baru/cleaning_result%20v1.csv'
df1 = pd.read_csv(df1, sep=",")
df = df1
df.head()

,full_text,text_clean,class
0,padahal nih yah ga usah pada banyak cingcong i...,perlu cingcong orang surabaya bandung medan ni...,0
1,Bisa naik jaklingko dari cipayung ke tamini 0 ...,naik jaklingko cipayung tamini rupiah wkwkwk l...,1
2,gw banget lagi terus naik jaklingko gratis ke ...,banget terus naik jaklingko gratis pelni,1
3,Ya mesti dibuat Stasiun baru karna Stasiun ter...,mesti buat stasiun baru stasiun dekat ancol ha...,2
4,Nilai tauladan yg bisa diambil dari nunggu jak...,nilai tauladan ambil nunggu jaklingko gin yaki...,0


In [ ]:
df['tweet'] = df['text_clean']
df['tweet']

0       perlu cingcong orang surabaya bandung medan ni...
1       naik jaklingko cipayung tamini rupiah wkwkwk l...
2                banget terus naik jaklingko gratis pelni
3       mesti buat stasiun baru stasiun dekat ancol ha...
4       nilai tauladan ambil nunggu jaklingko gin yaki...
                              ...                        
1195    transjakarta angkot jaklingko sekarang galak b...
1196    ombgunan tugu skali tiada kmanfaatanya warga p...
1197    transjakarta jarak angkot jaklingko masuk oper...
1198    nunggu jaklingko pulang kerja harap perintah m...
1199    halo betul nemu tweet cari cara pakai transpor...
Name: tweet, Length: 1200, dtype: object

In [ ]:
X = np.array(df['tweet'].tolist())
y = np.array(df['class'].tolist())

x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=10)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(960,)
(960,)
(240,)
(240,)


In [ ]:
className = {
    0: -1,
    1: 1,
    2: 2
}

def oneHotEncode(listArray):
    s = (len(listArray), len(np.unique(listArray)))
    tempData = np.zeros(s)
    for i in range(len(listArray)):
        if listArray[i] == 1:
            tempData[i][1] = 1
        elif listArray[i] == -1:
            tempData[i][0] = 1
        elif listArray[i] == 2:
            tempData[i][2] = 1
    return tempData

y_train = oneHotEncode(y_train)
y_test = oneHotEncode(y_test)

In [ ]:
max_vocab_length = 350
max_length = 33
text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)
text_vectorizer.adapt(X)

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 350
Top 5 most common words: ['', '[UNK]', 'jaklingko', 'naik', 'transjakarta']
Bottom 5 least common words: ['error', 'dua', 'depan', 'dana', 'cikini']


In [ ]:
embedding = layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    embeddings_initializer="uniform",
    input_length=max_length
)


In [ ]:
random_sentence = random.choice(x_train)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
transjakarta kartu flazz mikro trans angkot jaklingko untuk trans jakarta

Embedded version:


<tf.Tensor: shape=(1, 33, 128), dtype=float32, numpy=
array([[[ 0.04039698, -0.01490679,  0.0079907 , ..., -0.03696258,
         -0.04255539,  0.01015174],
        [ 0.04699235,  0.02622983,  0.02846067, ...,  0.00910113,
          0.02711561, -0.04153495],
        [-0.03557125,  0.01069098,  0.03911935, ..., -0.04436765,
          0.02677329, -0.03076104],
        ...,
        [ 0.02692716,  0.04234222,  0.04707935, ..., -0.03843965,
          0.03891351,  0.01781804],
        [ 0.02692716,  0.04234222,  0.04707935, ..., -0.03843965,
          0.03891351,  0.01781804],
        [ 0.02692716,  0.04234222,  0.04707935, ..., -0.03843965,
          0.03891351,  0.01781804]]], dtype=float32)>

In [ ]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(x_train)

X_train = tokenizer.texts_to_sequences(x_train)
X_train = pad_sequences(X_train, maxlen=max_length)

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(x)

out = tf.keras.layers.Dense(64, activation='relu')(x)
out = tf.keras.layers.Dense(16, activation='relu')(x)

out = tf.keras.layers.Dropout(0.5)(out)

outputs = layers.Dense(3, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    restore_best_weights=True,
    patience=3,
    min_delta=0.01,
    monitor='val_loss'
)

In [ ]:
model_history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=60,
    validation_data=(x_test, y_test),
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/20


ValueError: ignored

In [ ]:
import pandas as pd
import random
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Baca dataset
df1 = 'https://raw.githubusercontent.com/Zaniiiii/DataGemas/main/Baru/cleaning_result%20v1.csv'
df1 = pd.read_csv(df1, sep=",")
df = df1

# Kolom tweet sebagai fitur, kolom class sebagai target
X = df['text_clean'].values
y = df['class'].values

# Split data menjadi train set dan test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Menggunakan LabelEncoder untuk mengubah label menjadi angka
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

max_vocab_length = 350
max_length = 33

# Inisialisasi TextVectorization
text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

# Adaptasi TextVectorization pada data training
text_vectorizer.adapt(x_train)

# Inisialisasi model Sequential
model = Sequential()

# Menambahkan layer TextVectorization pada model
model.add(text_vectorizer)

# Menambahkan layer Embedding
model.add(layers.Embedding(input_dim=max_vocab_length + 1, output_dim=128, input_length=max_length))

# Menambahkan layer Conv1D
model.add(layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))

# Menambahkan layer MaxPooling1D
model.add(layers.MaxPooling1D(pool_size=2))

# Menambahkan layer LSTM
model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.35, return_sequences=True))

# Menambahkan layer Flatten
model.add(layers.Flatten())

# Menambahkan layer Dense
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='relu'))

# Menambahkan layer output dengan softmax activation untuk tiga kelas
model.add(layers.Dense(3, activation='softmax'))

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model
model.fit(x_train, y_train, epochs=60, batch_size=60, validation_data=(x_test, y_test), verbose=1)

# Melakukan prediksi pada data test
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

# Melakukan inverse transform pada label hasil prediksi untuk mendapatkan label awal
y_pred = label_encoder.inverse_transform(y_pred)
y_test = label_encoder.inverse_transform(y_test)

# Membuat laporan klasifikasi
classification_result = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_result)

Epoch 1/60
16/16 [==============================] - 3s 58ms/step - loss: 0.9079 - accuracy: 0.6740 - val_loss: 0.8230 - val_accuracy: 0.7042
Epoch 2/60
16/16 [==============================] - 1s 41ms/step - loss: 0.8177 - accuracy: 0.6833 - val_loss: 0.7923 - val_accuracy: 0.7042
Epoch 3/60
16/16 [==============================] - 1s 40ms/step - loss: 0.7852 - accuracy: 0.6833 - val_loss: 0.7959 - val_accuracy: 0.7042
Epoch 4/60
16/16 [==============================] - 1s 39ms/step - loss: 0.7203 - accuracy: 0.6885 - val_loss: 0.8278 - val_accuracy: 0.7042
Epoch 5/60
16/16 [==============================] - 1s 41ms/step - loss: 0.6253 - accuracy: 0.7385 - val_loss: 0.8474 - val_accuracy: 0.6958
Epoch 6/60
16/16 [==============================] - 1s 40ms/step - loss: 0.5504 - accuracy: 0.7583 - val_loss: 0.8483 - val_accuracy: 0.6917
Epoch 7/60
16/16 [==============================] - 1s 39ms/step - loss: 0.4989 - accuracy: 0.7833 - val_loss: 0.9283 - val_accuracy: 0.6792
Epoch 8/60
16

NameError: ignored

In [ ]:
max = 0
for i in range(1,20):

  # Baca dataset
  df1 = 'https://raw.githubusercontent.com/Zaniiiii/DataGemas/main/Baru/cleaning_result%20v1.csv'
  df1 = pd.read_csv(df1, sep=",")
  df = df1

  # Kolom tweet sebagai fitur, kolom class sebagai target
  X = df['text_clean'].values
  y = df['class'].values

  # Split data menjadi train set dan test set
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

  # Menggunakan LabelEncoder untuk mengubah label menjadi angka
  label_encoder = LabelEncoder()
  y_train = label_encoder.fit_transform(y_train)
  y_test = label_encoder.transform(y_test)

  max_vocab_length = 350
  max_length = 33

  # Inisialisasi TextVectorization
  text_vectorizer = TextVectorization(
      max_tokens=max_vocab_length,
      output_mode="int",
      output_sequence_length=max_length
  )

  # Adaptasi TextVectorization pada data training
  text_vectorizer.adapt(x_train)

  # Inisialisasi model Sequential
  model = Sequential()

  # Menambahkan layer TextVectorization pada model
  model.add(text_vectorizer)

  # Menambahkan layer Embedding
  model.add(layers.Embedding(input_dim=max_vocab_length + 1, output_dim=128, input_length=max_length))

  # Menambahkan layer Conv1D
  model.add(layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))

  # Menambahkan layer MaxPooling1D
  model.add(layers.MaxPooling1D(pool_size=2))

  # Menambahkan layer LSTM
  model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.35, return_sequences=True))

  # Menambahkan layer Flatten
  model.add(layers.Flatten())

  # Menambahkan layer Dense
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(16, activation='relu'))

  # Menambahkan layer output dengan softmax activation untuk tiga kelas
  model.add(layers.Dense(3, activation='softmax'))

  # Compile model
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Melatih model
  model.fit(x_train, y_train, epochs=60, batch_size=60, validation_data=(x_test, y_test), verbose=1)

  # Melakukan prediksi pada data test
  y_pred = model.predict(x_test)
  y_pred = np.argmax(y_pred, axis=1)

  # Melakukan inverse transform pada label hasil prediksi untuk mendapatkan label awal
  y_pred = label_encoder.inverse_transform(y_pred)
  y_test = label_encoder.inverse_transform(y_test)

  # Membuat laporan klasifikasi
  classification_result = classification_report(y_test, y_pred)
  print("Classification Report:")
  print(classification_result)

  _, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
  print('Akurasi pada data uji:', test_accuracy)

  if test_accuracy > max:
    max = test_accuracy

Epoch 1/60
16/16 [==============================] - 7s 97ms/step - loss: 0.9061 - accuracy: 0.6656 - val_loss: 0.8193 - val_accuracy: 0.7042
Epoch 2/60
16/16 [==============================] - 1s 71ms/step - loss: 0.8105 - accuracy: 0.6833 - val_loss: 0.7954 - val_accuracy: 0.7042
Epoch 3/60
16/16 [==============================] - 1s 69ms/step - loss: 0.7705 - accuracy: 0.6833 - val_loss: 0.7844 - val_accuracy: 0.7042
Epoch 4/60
16/16 [==============================] - 1s 70ms/step - loss: 0.7276 - accuracy: 0.6938 - val_loss: 0.8272 - val_accuracy: 0.7125
Epoch 5/60
16/16 [==============================] - 1s 71ms/step - loss: 0.6575 - accuracy: 0.7490 - val_loss: 0.7957 - val_accuracy: 0.6542
Epoch 6/60
16/16 [==============================] - 1s 71ms/step - loss: 0.5629 - accuracy: 0.7667 - val_loss: 0.7948 - val_accuracy: 0.6792
Epoch 7/60
16/16 [==============================] - 1s 69ms/step - loss: 0.4252 - accuracy: 0.8344 - val_loss: 0.8931 - val_accuracy: 0.6958
Epoch 8/60
16